In [2]:
%%writefile rename_files.py
import os
import urllib.parse
from pathlib import Path
import argparse

def rename_files(folder_path: Path,
                 file_pattern: str,
                 slice_start: int = 0) -> None:
    folder = Path(folder_path)

    if not folder.exists() or not folder.is_dir():
        print(f"Error: {folder} is not a valid folder.")
        return

    cnt = 0

    # Iterate deterministically for nicer logs
    for p in sorted(folder.glob(file_pattern)):
        # Skip directories
        if p.is_dir():
            continue

        original_name = p.name  # str
        # Decode URL-encoded characters
        decoded_name = urllib.parse.unquote(original_name)

        # Optionally remove first N characters (guard against over-slicing)
        if slice_start > 0 and len(decoded_name) > slice_start:
            decoded_name = decoded_name[slice_start:]

        # If name didn't change, skip
        if decoded_name == original_name:
            continue

        if decoded_name == "":
            print(f"⚠ Skipping (empty target name) for: {original_name}")
            continue

        new_path = p.with_name(decoded_name)

        # Prevent collisions
        if new_path.exists():
            print(f"⚠ Skipping (target exists): {decoded_name}")
            continue

        # Perform rename
        p.rename(new_path)
        print(f"Renamed: {original_name} → {decoded_name}")
        cnt += 1

    print(f"✅ Total files renamed: {cnt}")


if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Rename URL-encoded files in a folder")
    parser.add_argument("--folder_path", type=str, required=True, help="Path to the folder")
    parser.add_argument("--file_pattern", type=str, default="*", help="Glob to select files (e.g., '*.txt')")
    parser.add_argument("--slice_start", type=int, default=0,
                        help="Optional: remove first N characters from filename (after decoding)")

    args = parser.parse_args()
    rename_files(args.folder_path, args.file_pattern, args.slice_start)


Overwriting rename_files.py


In [1]:
!python3 rename_files.py --folder_path "/Users/AnirbanGuha/pCloud Drive/My Videos/Adult" --file_pattern "EPORNER*"

original name: EPORNERCOM20-20[HWfALK2ERBJ]20Ivy20Jons,20Kristin20Scott2020The20Real20Lesbian20Orgasmus20(1080).mp4
Decoded name: EPORNERCOM20-20[HWfALK2ERBJ]20Ivy20Jons,20Kristin20Scott2020The20Real20Lesbian20Orgasmus20(1080).mp4
Name didn't change....skipping
original name: EPORNERCOM20-20[LNoPErfXKhx]20JAPANESE20HOUSEWIFE20AND20STEPDAUGHTER20ARE20TIED20UP20RESTRAINED2020.mp4
Decoded name: EPORNERCOM20-20[LNoPErfXKhx]20JAPANESE20HOUSEWIFE20AND20STEPDAUGHTER20ARE20TIED20UP20RESTRAINED2020.mp4
Name didn't change....skipping
original name: EPORNERCOM20-20[OayTyz3d13U]20Kristen20Scott20The20Nerds20Revenge20(720).mp4
Decoded name: EPORNERCOM20-20[OayTyz3d13U]20Kristen20Scott20The20Nerds20Revenge20(720).mp4
Name didn't change....skipping
original name: EPORNERCOM20-20[Pdh8OWc0QAi]20Kristin2020Scott20Fuck20420BBC20(720).mp4
Decoded name: EPORNERCOM20-20[Pdh8OWc0QAi]20Kristin2020Scott20Fuck20420BBC20(720).mp4
Name didn't change....skipping
original name: EPORNERCOM20-20[aVQ1NnviG02]20Eng20Su

In [13]:
import os
import urllib.parse
from pathlib import Path
import argparse
import re

def rename_files(folder_path: Path,
                 file_pattern: str,
                 slice_start: int = 0) -> None:
    folder = Path(folder_path)

    if not folder.exists() or not folder.is_dir():
        print(f"Error: {folder} is not a valid folder.")
        return

    cnt = 0

    # Iterate deterministically for nicer logs
    for p in sorted(folder.glob(file_pattern)):
        # Skip directories
        if p.is_dir():
            continue

        original_name = p.name  # str
        # Decode URL-encoded characters
        # decoded_name = urllib.parse.unquote(original_name)
        # Replace all occurrences of 20 with a space
        decoded_name = re.sub(r'20', ' ', original_name)

        # Optional: collapse multiple spaces
        decoded_name = re.sub(r'\s+', ' ', decoded_name).strip()
        

        # Optionally remove first N characters (guard against over-slicing)
        if slice_start > 0 and len(decoded_name) > slice_start:
            decoded_name = decoded_name[slice_start:]

        # If name didn't change, skip
        if decoded_name == original_name:
            continue

        if decoded_name == "":
            print(f"⚠ Skipping (empty target name) for: {original_name}")
            continue

        new_path = p.with_name(decoded_name)

        # Prevent collisions
        if new_path.exists():
            print(f"⚠ Skipping (target exists): {decoded_name}")
            continue

        # Perform rename
        p.rename(new_path)
        print(f"Renamed: {original_name} → {decoded_name}")
        cnt += 1

    print(f"✅ Total files renamed: {cnt}")

In [14]:
folder_path = Path("/Users/AnirbanGuha/pCloud Drive/My Videos/Adult")
rename_files(folder_path, "EPORNER*", 27)

Renamed: EPORNERCOM20-20[HWfALK2ERBJ]20Ivy20Jons,20Kristin20Scott2020The20Real20Lesbian20Orgasmus20(1080).mp4 → Ivy Jons, Kristin Scott The Real Lesbian Orgasmus (1080).mp4
Renamed: EPORNERCOM20-20[LNoPErfXKhx]20JAPANESE20HOUSEWIFE20AND20STEPDAUGHTER20ARE20TIED20UP20RESTRAINED2020.mp4 → JAPANESE HOUSEWIFE AND STEPDAUGHTER ARE TIED UP RESTRAINED .mp4
Renamed: EPORNERCOM20-20[OayTyz3d13U]20Kristen20Scott20The20Nerds20Revenge20(720).mp4 → Kristen Scott The Nerds Revenge (7 ).mp4
Renamed: EPORNERCOM20-20[Pdh8OWc0QAi]20Kristin2020Scott20Fuck20420BBC20(720).mp4 → Kristin Scott Fuck 4 BBC (7 ).mp4
Renamed: EPORNERCOM20-20[aVQ1NnviG02]20Eng20SubYUJ-011-20I20was20seduced20by20a20married20colleague20and20dr.mp4 → Eng SubYUJ-011- I was seduced by a married colleague and dr.mp4
Renamed: EPORNERCOM20-20[eWC9aJNuIwm]20Paranormal202020Movie20(720).mp4 → Paranormal Movie (7 ).mp4
Renamed: EPORNERCOM20-20[gDv6VRPItg7]20[Eng20Sub]20On20That20Day,20At20That20Time,20In20That20Place,20Ever20.mp4 → [Eng Sub